In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
glove = {}
f = open('data/mini_glove.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    glove[word] = coefs
f.close()

In [4]:
class Glove2kSparse(nn.Module):
    def __init__(self):
        super(Glove2kSparse, self).__init__()
        self.encoder = nn.Linear(100, 1000)
        self.decoder = nn.Linear(1000, 100)
    
    def encode(self, x):
        x = self.encoder(x)
        topk = torch.topk(x, 7).indices
        debug_count = 0
        for index in range(len(x)):
            if index not in topk:
                x[index] = 0
                debug_count += 1
        return x
    
    def forward(self, x):  # Unsqueeze / squeeze?
        return self.decoder(self.encode(x))
    
    def view_grads(self):
        enc_grads = net.encoder.weight.grad.resize(100000)
        dec_grads = net.decoder.weight.grad.resize(100000)
        return sparse_vec_to_dict(torch.cat([enc_grads, dec_grads]))
        
net = Glove2kSparse()

In [5]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

for epoch in range(10):
    epoch_loss = 0
    for j in range(len(glove)):
        x_var = torch.tensor(list(glove.values())[j]).squeeze(0)
        optimizer.zero_grad()
        xpred_var = net(x_var)
        loss = criterion(xpred_var, x_var)
        loss.backward()
        optimizer.step()
        
        # running loss
        epoch_loss += loss.item()
        if j%1000 == 0 and j > 0:
            print(j, "   ", epoch_loss / j)
    print(f"Epoch {epoch} Average Loss:  {epoch_loss / 10000}")

1000     0.19631743665784598
2000     0.19271614674478768
3000     0.19105273923277855
4000     0.18984142135456206
5000     0.1884737504310906
6000     0.18714667668938637
7000     0.1864700799568423
8000     0.18611903664050625
9000     0.18680317832570936
Epoch 0 Average Loss:  0.184958866462484
1000     0.1490420922189951
2000     0.14962921951711178
3000     0.15072916285072763
4000     0.15079264387767763
5000     0.15034850658252835
6000     0.14982978206003705
7000     0.14971965642486298
8000     0.15013340625632554
9000     0.1517731101397011
Epoch 1 Average Loss:  0.15145419912971556
1000     0.13440479839593172
2000     0.1334765246156603
3000     0.1334320896329979
4000     0.13361731371469796
5000     0.13379363585039974
6000     0.13379630222978692
7000     0.13422396573317902
8000     0.13512390728713944
9000     0.13715997306298877
Epoch 2 Average Loss:  0.13738820332325996
1000     0.12469653276726604
2000     0.12465826020389795
3000     0.12509883635366956
4000     

In [7]:
# torch.save(net.state_dict(), "data/7_sparse_glove.pt")
# net.load_state_dict(torch.load("data/very_sparse_glove.pt"))

In [1]:
def word2sparse(word):
    return net.encode(torch.tensor(glove[word])).squeeze().detach().numpy()

def vec_dist(vec1, vec2):
    diff = vec1 - vec2
    return np.sqrt(np.dot(diff, diff))

def sparse_similarity(vec1, vec2):
    count = 0
    for index in range(len(vec1)):
        if vec1[index] != 0 and vec2[index] != 0:
            count += 1
    return count

def show_sparse(vec):
    output = {}
    for index in range(len(vec)):
        if vec[index] != 0:
            output[index] = vec[index]
    return output